# Optuna and MLFlow

In [ ]:
import optuna
import mlflow
from optuna.integration.mlflow import MLflowCallback

In [ ]:
# Hack to avoid unwanted logs from alembic
import logging
logging.getLogger('alembic.runtime.migration').disabled = True

## Setup MLFlowCallback

Default integration with MLFlow, logs parameters and objective result to MLFlow

In [ ]:
mlflc = MLflowCallback(
    tracking_uri="sqlite:///data/mlflow.db",
    metric_name="target",
    nest_trials=True
)

## Define Objective

In [ ]:
my_function = lambda x: (x - 2) ** 2

@mlflc.track_in_mlflow()
def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    mlflow.log_param("power", 2)
    mlflow.log_metric("base of metric", x - 2)

    return my_function(x)

## Optimize

In [ ]:
optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(study_name="my_study")

study.optimize(objective, n_trials=10, callbacks=[mlflc])
print(f"Minmal value is {study.best_value} at {study.best_params['x']}")

## View in MLFlow

Run the following command in the base folder of this project and click on the link

```bash
mlflow ui --backend-store-uri sqlite:///data/mlflow.db
```
